# 常规赛：PALM眼底彩照中黄斑中央凹定位比赛
> 基线分数：0.01698

> 改进不限于骨干网络，损失函数以及多损失函数策略等方式


**赛题介绍**

PALM黄斑定位常规赛的重点是研究和发展与患者眼底照片黄斑结构定位相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上定位黄斑的自动算法。具体目的是预测黄斑中央凹在图像中的坐标值。

![](https://ai-studio-static-online.cdn.bcebos.com/866fabe65cda42f497a05ab5123ad0f800b92b9e47904ed9b178943ab9e9cc56)

**数据简介**

PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带黄斑中央凹坐标标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。

**数据说明**

本次常规赛提供的金标准由中山大学中山眼科中心的7名眼科医生手工进行标注，之后由另一位高级专家将它们融合为最终的标注结果。本比赛提供数据集对应的黄斑中央凹坐标信息存储在xlsx文件中，名为“Fovea_Location_train”，第一列对应眼底图像的文件名(包括扩展名“.jpg”)，第二列包含x坐标，第三列包含y坐标。

![](https://ai-studio-static-online.cdn.bcebos.com/0f4bd2fb00cc4a4bb4d68ad03edcc101ae9f784d247f4267a316ae8489c3281b)

**训练数据集**

[数据集下载](https://bj.bcebos.com/v1/dataset-bj/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/%E5%B8%B8%E8%A7%84%E8%B5%9B%EF%BC%9APALM%E7%9C%BC%E5%BA%95%E5%BD%A9%E7%85%A7%E4%B8%AD%E9%BB%84%E6%96%91%E4%B8%AD%E5%A4%AE%E5%87%B9%E5%AE%9A%E4%BD%8D.zip)

文件名称：Train

* Train文件夹里有一个文件夹fundus_images和一个xlsx文件。

* fundus_images文件夹内包含800张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、P0001.jpg、N0001.jpg和V0001.jpg。
xlsx文件中包含800张眼底彩照对应的x、y坐标信息。

**测试数据集**

文件名称：PALM-Testing400-Images 文件夹里包含400张眼底彩照，命名形如T0001.jpg。

## 项目文件组织结构

- 常规赛：PALM眼底彩照中黄斑中央凹定位  ： 解压出的数据集文件夹

- utils  ：该项目中的一些数据方法和网络

	- dataset.py ：Dataset的加载
   
   - utils.py ：划分数据以及数据预/后处理的方法
   
   - models.py ：模型以及损失函数层

## 1.解压数据集

> 尽量解压到当前目录哦，方便一键式训练

In [ ]:
!unzip -oq /home/aistudio/data/data100477/常规赛：PALM眼底彩照中黄斑中央凹定位.zip

## 2.导入依赖

In [ ]:
import pandas as pd
import os
import time
import numpy as np
from PIL import Image,ImageDraw, ImageFilter

import paddle
from paddle import nn
from paddle import optimizer
from paddle import regularizer
from paddle import metric
from paddle.nn import loss
from paddle.nn import Layer

from paddle.io import Dataset, DataLoader
from paddle.vision import transforms

## 3.划分处理数据

根据`比赛数据集根目录`自动生成，训练、验证、测试的csv文件

> 一点修改的说明: 训练数据的坐标转化为0-1的相对值，方便模型预测

In [ ]:
from utils.utils import parse_data

# 比赛数据集根目录 -- 可自行指定路径
game_dataset_dir = '常规赛：PALM眼底彩照中黄斑中央凹定位'

# 开始解析数据，生成三个csv文件在最顶层目录，分别表示train、dev、test
# split_rate: 划分train的比例
parse_data(dir_path=game_dataset_dir, split_rate=0.8)

## 4.配置训练参数

In [ ]:
# 数据csv路径
train_path  = 'Fovea_Location_train.csv'
dev_path    = 'Fovea_Location_dev.csv'
test_path   = 'Fovea_Location_test.csv'

# 缩放目标大小
target_size = (224, 224)


# 配置任务参数
config = {
    'train':        pd.read_csv(train_path),   # 训练数据
    'dev':          pd.read_csv(dev_path),     # 验证数据
    'test':         pd.read_csv(test_path),    # 测试数据
    'num_point':    2,                         # 预测关键点个数
    'dropout_rate': 0.2,                       # drop率
    'target_size':  target_size,               # 目标大小
    'transform':    transforms.Compose([
                    transforms.Resize(target_size),
                    transforms.ToTensor(),
                    transforms.Normalize()
                    ]),
    'learn_rate':   4e-5,                      # 学习率
    'opt':          optimizer.AdamW,           # 优化方法
    'batch_size':   64,                        # 批大小
    'epoches':      344                       # 训练轮次
}

## 5.加载数据集

In [ ]:
# 导入任务数据集加载方法
from utils.dataset import Focus_Dataset
# 导入任务网络与损失函数
from utils.model   import FocusNet, FocusBCELoss
# 导入训练、验证、预测的接口
from utils.utils   import FocusTrain, FocusEvaluate, FocusPredict

# 加载数据集
train_dataset = Focus_Dataset(config=config, mode='train')
dev_dataset = Focus_Dataset(config=config, mode='dev')
test_dataset = Focus_Dataset(config=config, mode='test')

# 导入加载器
train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=config['batch_size'],
                              shuffle=True, drop_last=False)
dev_dataloader = DataLoader( dataset=dev_dataset, 
                             batch_size=config['batch_size'],
                             shuffle=True, drop_last=False)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=config['batch_size'],
                             shuffle=False, drop_last=False)

## 6.加载模型和训练参数

> 可在`utils/models.py`中修改**FocusNet**的骨干网络等

> 也可修改其中对应的损失函数`FocusBCELoss`层

In [ ]:
# 实例化模型
# 关键点个数：config['num_point']
# drop率：drop_rate=config['dropout_rate']
model = FocusNet(num_point=config['num_point'], drop_rate=config['dropout_rate'])
loss  = FocusBCELoss(weights=[0.7, 0.3])   # weights，不同类别的损失权重
 
# 训练的总迭代次数
total_iters = config['epoches'] * len(train_dataloader)

# 设置预热次数(百分之1的迭代次数用于预热)
warm_up_steps = config['epoches'] * len(train_dataloader) * 0.01
# 设置衰减次数（剩下的用于多项式衰减）
poly_decay_steps = total_iters - warm_up_steps

# 配置预热学习率策略器
Warming_up = optimizer.lr.LinearWarmup(learning_rate=config['learn_rate'], warmup_steps=warm_up_steps, start_lr=0, end_lr=config['learn_rate'])  # warmup_steps 这里在训练时，为iter迭代更新，非轮次更新
# 配置多项式衰减学习率策略器
Poly_decay = optimizer.lr.PolynomialDecay(learning_rate=config['learn_rate'], decay_steps=poly_decay_steps, power=0.9, end_lr=0.)

# 配置优化器
optimize = config['opt']

In [ ]:
# 加载之前训练的参数
# model.set_state_dict(paddle.load(path='models/Epoch_69_Loss_0.0593.pdparams'))

## 7.开始训练

In [6]:
# 开始训练，训练完成返回最后的模型
# eval_cycle对应验证的周期轮次，同时验证完后进行保存
model = FocusTrain(     focus_model=model,      # 参与训练的模型
                        train_dataloader=train_dataloader,   # 训练数据
                        dev_dataloader=dev_dataloader,       # 验证数据
                        epoches=config['epoches'],           # 训练总轮次
                        loss=loss,                           # 损失函数--损失层
                        optimize=optimize,                   # 优化方法
                        Warming_up=Warming_up,               # 添加预热学习率策略
                        Poly_decay=Poly_decay,               # 添加多项式衰减学习率策略
                        save_dir=None,                       # 保存路径,为None时，保存在models目录下
                        eval_cycle=5, 
                        vdl_writer=None,                     # 暂不支持
                        log_iter=5)                          # 迭代输出日志的周期，5次迭代输出一次信息

## 8.验证

In [7]:
FocusEvaluate(focus_model=model, loss=loss, dataloader=dev_dataloader)

## 9.预测与后处理

将预测的0-1的值，还原为实际坐标值

In [16]:
# 创建模型
model = FocusNet(num_point=2)
# 替换为训练好的模型参数路径即可
model.set_state_dict(paddle.load(path='models/Epoch_359_Loss_0.0161.pdparams'))

# 预测结果——后处理，得到提交结果--保存在首目录中-Fovea_Localization_Results.csv
# 并返回保存的csv，以供检查是否有误
submit_df = FocusPredict(focus_model=model, dataloader=test_dataloader)
submit_df

# 结果投票集成
* 简单投票集成学习，这个可以提升效果，尽量选得分高的进行投票。
* 在统计学和机器学习中，集成学习方法使用多种学习算法来获得比单独使用任何单独的学习算法更好的预测性能。
* 使用不同超参数、不同的特征，不同的结构，运行多次模型可得到不同的预测结果。在这里我使用的是简单投票法，取PM Risk 平均值作为最终的预测结果。 预测出多个result后，进行投票，
* 榜单上的成绩，便是用各个预测结果进行投票生成的

In [17]:
%cd /home/aistudio/
import pandas as pd

import numpy as np
df1 = pd.read_csv("0.01688.csv")
df2 = pd.read_csv("0.01692.csv")


dfs=[df1,df2]
Fovea_X=[]
File_Name=[]
Fovea_Y=[]

for i in range(len(df1)):
    File_Name.append(dfs[0]['FileName'][i])
    avg=(sum(np.array(dfs[x]['Fovea_X'][i]) for x in range(len(dfs))))/len(dfs)
    avg1=(sum(np.array(dfs[y]['Fovea_Y'][i]) for y in range(len(dfs))))/len(dfs)
    
    PM_Risk.append(avg)
    Fovea_Y.append(avg1)

submission = pd.DataFrame(data={
                            "FileName": File_Name,
                            "Fovea_X": Fovea_X,
                            "Fovea_Y": Fovea_Y
                        })
submission=submission.sort_values(by='FileName')
submission.to_csv("Fovea_Localization_Results.csv", index=False)

/home/aistudio


# 总结
* 投票器效果并不明显
* 得到0.1692迭代次数太多，相对于基线优化效果差
* 可适当增大输入尺寸